In [22]:
%%capture
!pip install yahooquery

In [2]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker
from sklearn.linear_model import LinearRegression

In [254]:
class CapitalStock:
    
    def __init__(self, symbol, company, details=['FreeCashFlow', 'StockholdersEquity', 'TotalDebt', 'CapitalStock'], period='max', interval='1d', start=None, end=None):
        
        self.symbol = symbol
        self.Tick = Ticker(symbol, trailing=False)
        self.company = company  
        
        self.FinancialData = self.Tick.get_financial_data(details, trailing=False)
        self.HistoricalData = self.Tick.history(period, interval, start, end)
        
    
    def BetaRisk(self, span='1y', typed=False):
        """
        Requires packages yahooquery and sklearn.linear_model.
        Returns the beta risk facotr of the stock against the S&P/ASX 200 index. 
        Span kwarg based on yahooquery period kwargs. See: https://yahooquery.dpguthrie.com/guide/ticker/historical/
        """
        exchange_index = Ticker('^AXJO').history(period=span)
        df = pd.concat([self.HistoricalData, exchange_index]).unstack(level=0)
        df_adjclose = df['adjclose'].pct_change()
        df_adjclose = df_adjclose.dropna()
        
        X = np.array(df_adjclose['^AXJO']).reshape((-1,1))
        y = np.array(df_adjclose[self.symbol])

        model = LinearRegression().fit(X, y)

        if typed == True:
            return print(self.company, 'Beta: ', model.coef_[0])

        return model.coef_[0]
    
    def Wacc(self, span='1y'):
        
        MARKET_RETURN = 0.132 #Source: https://files.marketindex.com.au/files/statistics/historical-returns-infographic-2022.pdf
        RISK_FREE_RATE = float(Ticker('^IRX', trailing=False).history(period='7d')['close'].iloc[-1]) / 100 #Returns previous day's close price of 13 week US treasury bond
        COD = 0.1357 #Source: https://www.gurufocus.com/term/wacc/ASX:S32/WACC-/South32-Ltd

        Equity = self.FinancialData['StockholdersEquity'][-1]
        Debt = self.FinancialData['TotalDebt'][-1]
        beta = self.BetaRisk()
        
        weight_average_cost_of_capital = Equity / (Debt + Equity)  * (RISK_FREE_RATE + beta * (MARKET_RETURN - RISK_FREE_RATE)) + Debt / (Debt + Equity) * COD
        
        return weight_average_cost_of_capital
    
    def NetPresentValue(self, long_term_rate=0.1, years=10, span='1y'):
        free_cash_flow = self.FinancialData['FreeCashFlow'].iloc[-1]
 
        compound_rate = (self.FinancialData['FreeCashFlow'].iloc[-1] / self.FinancialData['FreeCashFlow'].iloc[0]) ** (1 / len(self.FinancialData)) - 1
        discount_rate = self.Wacc(span)

        present_value = np.sum([free_cash_flow * ((1 + compound_rate) ** i) / ((1 + discount_rate) ** i) for i in range(years)])
        last_free_cash_flow =  free_cash_flow * ((1 + compound_rate) ** (years)) / ((1 + discount_rate) ** (years))

        terminal_value = last_free_cash_flow * (1 + long_term_rate) / ((discount_rate - long_term_rate)  * (1 + discount_rate) ** (years + 1))
 
        current_stock_value = (present_value + terminal_value) / self.FinancialData['CapitalStock'].iloc[-1] * 1.46 

        return current_stock_value
        

In [255]:
s32 = CapitalStock('S32.AX', 'South32', period='5y')

In [256]:
s32.NetPresentValue(years=10, span='1y')

4.43750558794623

In [ ]:
s32.NetPresentValue()

0.23572285133991455
0.15771726776960862


(17068279809.10266, 4449044984.522774)

In [124]:
s32.BetaRisk()

1.3473960087468237

In [126]:
s32.Wacc(span='1y')

0.15769646889887676

In [51]:
s32.FinancialData

,asOfDate,periodType,currencyCode,CapitalStock,FreeCashFlow,StockholdersEquity,TotalDebt
symbol,,,,,,,
S32.AX,2019-06-30,12M,USD,1.421200e+10,1.061000e+09,1.016900e+10,9.040000e+08
S32.AX,2020-06-30,12M,USD,1.394300e+10,5.920000e+08,9.563000e+09,1.017000e+09
S32.AX,2021-06-30,12M,USD,1.359700e+10,8.140000e+08,8.955000e+09,1.207000e+09
S32.AX,2022-06-30,12M,USD,1.346900e+10,2.474000e+09,1.078000e+10,1.827000e+09


In [56]:
s32.HistoricalData.head()

open  high    low  close    volume  adjclose
symbol date                                                    
S32.AX 2023-01-03  4.02  4.03  3.910   3.94  10515163      3.94
       2023-01-04  3.99  4.07  3.980   4.07  12362739      4.07
       2023-01-05  4.12  4.13  4.060   4.07  13388326      4.07
       2023-01-06  4.11  4.26  4.110   4.25  18456584      4.25
       2023-01-09  4.38  4.45  4.365   4.41  20074007      4.41